# Importar librerias

In [103]:
# review current interpeter

import sys
import pandas as pd
print(sys.executable)

/Users/joseovalle/Desktop/mlops_jovalle/.venv/bin/python


# Manipulación y prepración de datos
* Importar los conjuntos de datos con los que se trabajará.
* Realizar tareas de limpieza de datos, eliminando valores nulos o inconsistentes.
* Realizar transformaciones necesarias para preparar los datos para el análisis y modelado.

Importar Datos

In [102]:

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
regensburg_pediatric_appendicitis = fetch_ucirepo(id=938) 
  
# data (as pandas dataframes) 
X = regensburg_pediatric_appendicitis.data.features 
y = regensburg_pediatric_appendicitis.data.targets 
  
# metadata 
print(regensburg_pediatric_appendicitis.metadata) 
  
# variable information 
print(regensburg_pediatric_appendicitis.variables) 


FileNotFoundError: [Errno 2] No such file or directory

Realizar tareas de limpieza de datos, eliminando valores nulos o inconsistentes.

In [ ]:
# Revisar información de las variables

features_df = regensburg_pediatric_appendicitis.variables
non_categorical_df = features_df[features_df['type'] == 'Continuous']
non_categorical_features = non_categorical_df['name'].unique()
non_categorical_features = list(non_categorical_features)
non_categorical_features

['Age',
 'BMI',
 'Height',
 'Body_Temperature',
 'WBC_Count',
 'RBC_Count',
 'Hemoglobin',
 'RDW']

In [ ]:
# calculate null values
data_nulls = X.isnull().sum()
df_nulls = pd.DataFrame(data_nulls, columns = ['Nulls'])
df_nulls.reset_index(inplace = True)
df_nulls.rename(columns = {'index':'Feature'}, inplace = True)
df_nulls. head()

,Feature,Nulls
0,Age,1
1,BMI,27
2,Sex,2
3,Height,26
4,Weight,3


There are null values for categorial variables which seem to indicate the lack of that categorical variable but not related to a null value

In [ ]:
# calculate % of nulls values
records = len(X)
df_percentage_nulls = df_nulls
df_percentage_nulls['total_recordsrecords'] = records
df_percentage_nulls['nulls_percent(%)'] = df_percentage_nulls['Nulls'] / records
df_percentage_nulls.head()
df_percentage_nulls.sort_values(by= 'nulls_percent(%)', ascending= False).head(10)

,Feature,Nulls,records,nulls_percent(%),total_recordsrecords
43,Abscess_Location,769,782,0.983376,782
52,Gynecological_Findings,756,782,0.966752,782
47,Conglomerate_of_Bowel_Loops,739,782,0.945013,782
19,Segmented_Neutrophils,728,782,0.930946,782
48,Ileus,722,782,0.923274,782
39,Perfusion,719,782,0.919437,782
51,Enteritis,716,782,0.915601,782
38,Appendicolith,713,782,0.911765,782
49,Coprostasis,711,782,0.909207,782
40,Perforation,701,782,0.896419,782


In [ ]:
# calculate null values for continuous variables
filtered_df = df_percentage_nulls[df_percentage_nulls['Feature'].isin(non_categorical_features)]
filtered_df


,Feature,Nulls,records,nulls_percent(%),total_recordsrecords
0,Age,1,782,0.001279,782
1,BMI,27,782,0.034527,782
3,Height,26,782,0.033248,782
16,Body_Temperature,7,782,0.008951,782
17,WBC_Count,6,782,0.007673,782
21,RBC_Count,18,782,0.023018,782
22,Hemoglobin,18,782,0.023018,782
23,RDW,26,782,0.033248,782


Replacing null values for continous variables

In [ ]:
X.head()

,Age,BMI,Sex,Height,Weight,Length_of_Stay,Alvarado_Score,Paedriatic_Appendicitis_Score,Appendix_on_US,Appendix_Diameter,...,Abscess_Location,Pathological_Lymph_Nodes,Lymph_Nodes_Location,Bowel_Wall_Thickening,Conglomerate_of_Bowel_Loops,Ileus,Coprostasis,Meteorism,Enteritis,Gynecological_Findings
0,12.68,16.9,female,148.0,37.0,3.0,4.0,3.0,yes,7.1,...,NaN,yes,reUB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14.10,31.9,male,147.0,69.5,2.0,5.0,4.0,no,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,NaN,NaN
2,14.14,23.3,female,163.0,62.0,4.0,5.0,3.0,no,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,NaN
3,16.37,20.6,female,165.0,56.0,3.0,7.0,6.0,no,NaN,...,NaN,yes,reUB,NaN,NaN,NaN,NaN,NaN,yes,NaN
4,11.08,16.9,female,163.0,45.0,3.0,5.0,6.0,yes,7.0,...,NaN,yes,reUB,NaN,NaN,NaN,NaN,NaN,yes,NaN


In [ ]:
# Iterar sobre variables no categoricas e imputar con la mediana

for feature in non_categorical_features:
    # Calculate the median of the column (excluding NaN values)
    median_value = X[feature].median()    
    # Fill NaN values with the median of the column
    X.loc[:, feature] = X[feature].fillna(median_value)

In [ ]:
# Validar si aun existen datos nulos
X[non_categorical_features].isnull().sum()

Age                 0
BMI                 0
Height              0
Body_Temperature    0
WBC_Count           0
RBC_Count           0
Hemoglobin          0
RDW                 0
dtype: int64

# Exploración y preprocesamiento de datos
* Realizar análisis exploratorio de datos (EDA) utilizando visualizaciones y estadísticas descriptivas.
* Aplicar técnicas de preprocesamiento como normalización, codificación de variables categóricas y reducción de dimensionalidad.


# Versionado de datos

* Utilizar herramientas de versionado de datos para mantener un registro de cambios y versiones de los conjuntos de datos utilizados.
* Documentar todas las modificaciones realizadas en los datos.

# Construcción, ajuste y evaluación de modelos de machine learning
* Seleccionar algoritmos de Machine Learning adecuados para resolver el problema planteado.
* Entrenar los modelos utilizando los datos preprocesados.
* Realizar ajustes de hiperparámetros para optimizar el rendimiento del modelo.
* Evaluar los modelos utilizando métricas de rendimiento específicas.


# Aplicación de mejores prácticas en el pipeline de modelado
* Implementar pipelines de Machine Learning en donde se automaticen las tareas de preprocesamiento, entrenamiento y evaluación.
* Documentar cada paso del pipeline para asegurar claridad y reproducibilidad.


# Estructuración y refactorización del código
* Organizar el código en módulos y funciones bien definidas.
* Aplicar principios de programación orientada a objetos para mejorar la estructura del código.
* Refactorizar el código existente para mejorar su eficiencia y mantenibilidad.

# Presentación ejectuvia de pase pdf